# Introduction 

This note evaluate phi3-medium model with llama.cpp

# Env and tools prepare

In [1]:
!cp -r /kaggle/input/llama-cpp-binary-compiled-with-gpu/llama.cpp /kaggle/working/

In [2]:
!chmod +x -R llama.cpp
%cd llama.cpp
!cp -r /usr/local/cuda-12.3/targets /usr/local/nvidia/ 

/kaggle/working/llama.cpp


# Run perplexity

Perplexity is the main evaluation metric for large language models (LLMs). It measures how well the model predicts a given sequence of tokens.
Formally, the perplexity of LLMs is the exponentiated average negative log-likelihood,  which means **perplexity smaller is better** 

Perplexity benchmark shows how accurate the model is 

In [3]:

!/kaggle/working/llama.cpp/llama-perplexity -m  /kaggle/input/phi3-medium-128k-instruct-gguf/gguf/fp16/1/phi3-medium-128k-instruct.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > ppl_fp16.log 2>&1
!/kaggle/working/llama.cpp/llama-perplexity -m  /kaggle/input/phi3-medium-128k-instruct-gguf/gguf/q4_k_m/1/phi3-medium-128k-instruct-Q4_K_M-v2.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > ppl_Q4_v2.log 2>&1
!/kaggle/working/llama.cpp/llama-perplexity -m  /kaggle/input/phi3-medium-128k-instruct-gguf/gguf/q4_k_m/1/phi3-medium-128k-instruct-Q4_K_M.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > ppl_Q4.log 2>&1


In [4]:
!tail -6 ppl_fp16.log
!tail -6 ppl_Q4_v2.log
!tail -6 ppl_Q4.log

Final estimate: PPL = 4.1981 +/- 0.03582

llama_perf_context_print:        load time =  260727.61 ms
llama_perf_context_print: prompt eval time = 2157650.93 ms / 131072 tokens (   16.46 ms per token,    60.75 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time = 2172436.78 ms / 131073 tokens
Final estimate: PPL = 4.4018 +/- 0.03803

llama_perf_context_print:        load time =   64308.35 ms
llama_perf_context_print: prompt eval time = 1147874.80 ms / 131072 tokens (    8.76 ms per token,   114.19 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time = 1163920.29 ms / 131073 tokens
Final estimate: PPL = 4.4018 +/- 0.03803

llama_perf_context_print:        load time =   67686.25 ms
llama_perf_context_print: prompt eval tim

# Benchmarking the Inference Throughput and Memory Consumption 

Llama-bench shows performance and resource consumption. 
There are three types of test
- Prompt processing (pp): processing a prompt in batches (-p)
- Text generation (tg): generating a sequence of tokens (-n)
- Prompt processing + text generation (pg): processing a prompt followed by generating a sequence of tokens (-pg)

In the following test, we run text generation and prompt processing


In [5]:
!/kaggle/working/llama.cpp/llama-bench -m /kaggle/input/phi3-medium-128k-instruct-gguf/gguf/fp16/1/phi3-medium-128k-instruct.gguf -m /kaggle/input/phi3-medium-128k-instruct-gguf/gguf/q4_k_m/1/phi3-medium-128k-instruct-Q4_K_M-v2.gguf -m /kaggle/input/phi3-medium-128k-instruct-gguf/gguf/q4_k_m/1/phi3-medium-128k-instruct-Q4_K_M.gguf -mg 1 -ngl 10,20,30,35 -n 128,256,512 

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: Tesla P100-PCIE-16GB, compute capability 6.0, VMM: yes
| model                          |       size |     params | backend    | ngl |   main_gpu |          test |                  t/s |
| ------------------------------ | ---------: | ---------: | ---------- | --: | ---------: | ------------: | -------------------: |
| phi3 14B F16                   |  26.00 GiB |    13.96 B | CUDA       |  10 |          1 |         pp512 |         76.09 ± 0.28 |
| phi3 14B F16                   |  26.00 GiB |    13.96 B | CUDA       |  10 |          1 |         tg128 |          0.99 ± 0.00 |
| phi3 14B F16                   |  26.00 GiB |    13.96 B | CUDA       |  10 |          1 |         tg256 |          0.99 ± 0.00 |
| phi3 14B F16                   |  26.00 GiB |    13.96 B | CUDA       |  10 |          1 |         tg512 |          0.98 ± 0.00 |
| phi3 14B F16   